### 切换工作路径

In [ ]:
!pwd

In [ ]:
import os
os.chdir('./libai')

### 读取loss数据

In [ ]:
import os
txt_list = sorted(os.listdir("./loss_txt"))
#print(txt_list)

### 绘制曲线

In [ ]:
#!pip install matplotlib

for i in range(0, len(txt_list), 2):
    file_name1 = txt_list[i].split('.')[-2]
    commit1 = file_name1.split('_')[-1]
    
    file_name2 = txt_list[i+1].split('.')[-2]
    commit2 = file_name2.split('_')[-1]
    
    # make commit2=='master'
    if commit1 == "master":
        file_tmp = file_name1
        commit_tmp = commit1
        file_name1 = file_name2
        commit1 = commit2
        file_name2 = file_tmp
        commit2 = commit_tmp
    
    #!echo $file_name1 $file_name2 $commit1 $commit2
    
    !sed -i "s#loss1.txt#loss_txt/{file_name1}.txt#g" draw_loss.py
    !sed -i "s#loss1#{file_name1}#g" draw_loss.py
    
    !sed -i "s#loss2.txt#loss_txt/{file_name2}.txt#g" draw_loss.py
    !sed -i "s#loss2#{file_name2}#g" draw_loss.py
    
    !sed -i "s#loss_curve.png#curve/loss_curve_{file_name1}.png#g" draw_loss.py
    !python3 draw_loss.py
    !sed -i "s#curve/loss_curve_{file_name1}.png#loss_curve.png#g" draw_loss.py
    
    !sed -i "s#loss_txt/{file_name1}.txt#loss1.txt#g" draw_loss.py
    !sed -i "s#{file_name1}#loss1#g" draw_loss.py
    
    !sed -i "s#loss_txt/{file_name2}.txt#loss2.txt#g" draw_loss.py
    !sed -i "s#{file_name2}#loss2#g" draw_loss.py

### 生成吞吐&显存的表格

In [ ]:
import os
os.chdir('../')

In [ ]:
#!pip install pyyaml
!python3 extract_libai_libai.py --compare-log ./libai/test_logs_loss/master/ --test-log ./libai/test_logs_loss/3d5e919/

In [ ]:
from IPython.display import Markdown, display

display(Markdown("./libai/test_logs_loss/master/dlperf_result.md"))

### 每个模型的多张曲线图合并成一张

In [ ]:
import os
os.chdir('./libai')
!mkdir img_bert img_gpt img_t5
!cd curve && cp loss_curve_LibAI_bert* ../img_bert/ && cp loss_curve_LibAI_gpt2* ../img_gpt/ && cp loss_curve_LibAI_t5* ../img_t5/

In [ ]:
for title in ['bert', 'gpt', 't5']:
    #!echo $title
    !sed -i "s#model#{title}#g" compose.py
    if title == 'gpt':
        !sed -i "s#IMAGE_COLUMN = 3#IMAGE_COLUMN = 2#g" compose.py
    !python3 compose.py
    !sed -i "s#{title}#model#g" compose.py
    !sed -i "s#IMAGE_COLUMN = 2#IMAGE_COLUMN = 3#g" compose.py

### 展示loss对齐曲线图

In [ ]:
from PIL import Image
from IPython.display import Image as IMG

display(Image.open('./curve/compose_bert.png'))

In [ ]:
display(Image.open('./curve/compose_gpt.png'))

In [ ]:
display(Image.open('./curve/compose_t5.png'))

In [ ]:
!~/ossutil64 -c ~/ossutilconfig cp -r -f ./curve/ oss://oneflow-test/loss_matching/GradAcc/oneflow-28/jupyter_test/curve/
!~/ossutil64 -c ~/ossutilconfig cp -r -f ./loss_txt/ oss://oneflow-test/loss_matching/GradAcc/oneflow-28/jupyter_test/loss_txt/
!~/ossutil64 -c ~/ossutilconfig cp -r -f ./test_logs_loss/ oss://oneflow-test/loss_matching/GradAcc/oneflow-28/jupyter_test/test_logs_loss/

In [ ]:
!rm -rf ./img_bert/
!rm -rf ./img_gpt/
!rm -rf ./img_t5/